In [26]:
# Nov. 11, 2021, Xiaole Zhang
# Download meteo data from ecmwf 
import cdsapi
from datetime import date, timedelta
import os
from pathlib import Path
import calendar 

# covert number to formated string
def number2String(f):
    return '%02d' % f

c = cdsapi.Client() 

# [up, left, down, right]
regionName = 'Beijing'
regionBound = {'Beijing': [50, 105, 30, 127],\
               'Asia': [55, 60, 0, 150]}

# data for Beijing
targetYear = 2021

targetMonths = [3]
monthsList = list(map(number2String, targetMonths))

targetDays = range(1,32)
daysList = list(map(number2String, targetDays))

# to download the meteo day by month to meet the volume requirement
for currentMonth in targetMonths:
    currentDay = date(targetYear, currentMonth, 1) 
    initalDay = date(targetYear, currentMonth, 1) 
    
    year = '%4d' % currentDay.year
    month = '%02d' % currentDay.month
    day = '%02d' % currentDay.day
    print('Current Day:' + currentDay.isoformat())
    
    # download single level data
    fileName = regionName + '/'+'SingleLevel_'+currentDay.strftime('%Y%m%d')+'.grib'
    filePath = Path(regionName + '/')
    filePath.mkdir(parents=True, exist_ok=True)
    c.retrieve( 
        'reanalysis-era5-single-levels', 
        { 
            'product_type': 'reanalysis', 
            'format': 'grib', 
            'variable': [ 
                '2m_temperature', 'boundary_layer_height', 'eastward_turbulent_surface_stress', 
                'evaporation', 'high_cloud_cover', 'medium_cloud_cover', 
                'northward_turbulent_surface_stress', 'skin_temperature', 'surface_sensible_heat_flux', 
                'surface_solar_radiation_downwards', 'surface_pressure', 'convective_precipitation','large_scale_precipitation','volumetric_soil_water_layer_1', 
            ], 
            'year': year, 
            'month': month,
            'day': daysList, 
            'time': [ 
                '00:00', '01:00', '02:00', 
                '03:00', '04:00', '05:00', 
                '06:00', '07:00', '08:00', 
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ], 
            'area': regionBound[regionName], 
        }, 
        fileName) 
    
    # split the data to daily files
    ruleFile =regionName+'/rules_file'
    os.system("grib_filter "+ruleFile+" "+fileName)
    
    ###############################################
    # download 3d data
    fileName3d =  regionName + '/'+'PressureLevel_'+currentDay.strftime('%Y%m%d')+'.grib'
    c.retrieve( 
        'reanalysis-era5-pressure-levels', 
        { 
        'product_type': 'reanalysis', 
        'variable': [ 
        'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 'specific_humidity', 
        'temperature', 'u_component_of_wind', 'v_component_of_wind', 
        ], 
        'pressure_level': [ 
        '500', '550', '600', 
        '650', '700', '750', 
        '775', '800', '825', 
        '850', '875', '900', 
        '925', '950', '975', 
        '1000', 
        ], 
        'year': year, 
        'month': month,
        'day': daysList, 
        'time': [ 
        '00:00', '01:00', '02:00', 
        '03:00', '04:00', '05:00', 
        '06:00', '07:00', '08:00', 
        '09:00', '10:00', '11:00',
        '12:00', '13:00', '14:00',
        '15:00', '16:00', '17:00',
        '18:00', '19:00', '20:00',
        '21:00', '22:00', '23:00', 
        ], 
        'area': regionBound[regionName], 
        'format': 'grib', 
        }, 
        fileName3d) 
    
    # split the data to daily files
    ruleFile =regionName+'/rules_file_pressure'
    os.system("grib_filter "+ruleFile+" "+fileName3d)
    
    ###########################
    # combine the daily 2d and 3d data files into single files and delete the splitted data
    # need to install the grib api tools
    # sudo apt-get install libgrib-api-tools

    dayRange = calendar.monthrange(targetYear, currentMonth)
    for day in range(dayRange[0], dayRange[1]):
        currentDay = initalDay + timedelta(days=day) 
        print('Current Day:' + currentDay.isoformat())
        fileName3d = regionName + '/' + 'ecmf_'+currentDay.strftime('%Y%m%d')+'_pressureLevels.grib'
        fileName2d = regionName + '/' + 'ecmf_'+currentDay.strftime('%Y%m%d')+'.grib'
        fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
        os.system("cat "+fileName2d+" "+fileName3d+" > "+fileNameAll)
        os.system("rm "+fileName2d+" "+fileName3d)    
   #fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
   #os.system("cat "+fileName+" "+fileName3d+" > "+fileNameAll)
   #os.system("rm "+fileName+" "+fileName3d)

2021-11-11 19:54:24,038 INFO Welcome to the CDS
2021-11-11 19:54:24,038 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-11-11 19:54:24,088 INFO Request is queued


Current Day:2021-03-01


2021-11-11 21:14:49,615 INFO Request is running
2021-11-11 21:18:50,011 INFO Request is completed
2021-11-11 21:18:50,012 INFO Downloading https://download-0011.copernicus-climate.eu/cache-compute-0011/cache/data5/adaptor.mars.internal-1636661641.899889-6243-1-174ebdfd-4417-42c2-bc4b-172b575d4664.grib to Beijing/SingleLevel_20210301.grib (145.4M)
2021-11-11 21:19:02,697 INFO Download rate 11.5M/s 
2021-11-11 21:19:04,840 INFO Welcome to the CDS
2021-11-11 21:19:04,841 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2021-11-11 21:19:04,876 INFO Request is queued
2021-11-11 22:15:28,325 INFO Request is running
2021-11-11 22:55:32,213 INFO Request is completed
2021-11-11 22:55:32,215 INFO Downloading https://download-0013.copernicus-climate.eu/cache-compute-0013/cache/data3/adaptor.mars.internal-1636665284.4785912-22932-8-032ec890-2fef-4da8-8544-6aed1d0efd9d.grib to Beijing/PressureLevel_20210301.grib (915.5M)
2021-11-11 22:57:03,

Current Day:2021-03-01
Current Day:2021-03-02
Current Day:2021-03-03
Current Day:2021-03-04
Current Day:2021-03-05
Current Day:2021-03-06
Current Day:2021-03-07
Current Day:2021-03-08
Current Day:2021-03-09
Current Day:2021-03-10
Current Day:2021-03-11
Current Day:2021-03-12
Current Day:2021-03-13
Current Day:2021-03-14
Current Day:2021-03-15
Current Day:2021-03-16
Current Day:2021-03-17
Current Day:2021-03-18
Current Day:2021-03-19
Current Day:2021-03-20
Current Day:2021-03-21
Current Day:2021-03-22
Current Day:2021-03-23
Current Day:2021-03-24
Current Day:2021-03-25
Current Day:2021-03-26
Current Day:2021-03-27
Current Day:2021-03-28
Current Day:2021-03-29
Current Day:2021-03-30
Current Day:2021-03-31
